In [4]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [12]:
# An attempt to use individual player performance from past years
# to predict NA LCS team standings in Spring 2020.

#lcs16 = pd.read_excel('2016 complete match data OraclesElixir 2018-12-18.xlsx')
#lcs17 = pd.read_excel('2017matchdataOraclesElixir.xlsx')
#spring_18 = pd.read_excel('2018-spring-match-data-OraclesElixir-2018-05-20.xlsx')
#summer_18 = pd.read_excel('2018 summer match data OraclesElixir 2018-09-27.xlsx')
#worlds_18 = pd.read_excel('2018-worlds-match-data-OraclesElixir-2018-11-03.xlsx')

spring_19 = pd.read_excel('2019-spring-match-data-OraclesElixir-2019-05-21.xlsx')
summer_19 = pd.read_excel('2019-summer-match-data-OraclesElixir-2019-09-16.xlsx')
#worlds_19 = pd.read_excel('League of Legends/2019-summer-match-data-OraclesElixir-2019-11-10.xlsx')
print('Done.')

Done.


In [13]:
lcs_19 = spring_19.append(summer_19)

In [14]:
model = LogisticRegression()

In [15]:
nalcs = lcs_19[lcs_19['league'] == 'LCS']
not_na = lcs_19[lcs_19['player'].isin(['Jiizuke', 'ry0ma', 'Kobbe', 'Broxah', 'sOAZ', 'IgNar'])]
new_na = nalcs.append(not_na)

In [16]:
tl = ['Doublelift', 'Impact', 'Jensen', 'CoreJJ', 'Broxah']
thieves = ['Ssumday', 'Meteos', 'Cody Sun', 'Stunt', 'ry0ma']
tsm = ['Broken Blade' , 'Dardoch', 'Kobbe', 'Bjergsen', 'Biofrost']
imt = ['sOAZ', 'Xmithie', 'Altec', 'Hakuho']
fq = ['V1per', 'Santorin', 'PowerOfEvil', 'WildTurtle', 'IgNar']
gg = ['Hauntzer', 'Closer', 'Goldenglue', 'FBI', 'Keith']
eg = ['Kumo', 'Svenskeren', 'Zeyzal', 'Bang', 'Jiizuke']
clg = ['Ruin', 'Wiggily', 'Crown', 'Stixxay', 'Smoothie']
c9 = ['Licorice', 'Blaber', 'Nisqy', 'Zven', 'Vulcan']
dig = ['Huni', 'Grig', 'Froggen', 'Johnsun', 'aphromoo']

na = [tl, thieves, tsm, imt, fq, gg, eg, clg, c9, dig]

all_players = list(lcs_19['player'].unique())

all = []
for team in na:
    played = []
    for player in team:
        if player in all_players:
            played.append([True, player])
        else:
            played.append([False, player])
    all.append(played)

all_numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = new_na.select_dtypes(include=all_numerics)
y = new_na['result']
x = numeric.drop(labels=['patchno', 'playerid', 'gamelength', 'fdtime', 'fttime', 'fbarontime', 'result', 'herald', 'heraldtime', 'fbaron'], axis=1)

good = ['k', 'a', 'teamtowerkills', 'teambaronkills', 'gspd', 'teamkills', 'teamdragkills']
bad = ['d', 'opptowerkills', 'oppbaronkills', 'teamdeaths', 'oppdragkills']

new_na = new_na.fillna(0)

In [17]:
# 1. use a logistic regression to see coefficients of good and bad features
# 2. aggregate a player's features weighted with coeffiecients
# 3. aggregate the features of all players on a team
# 4. use aggregate team scores to rank teams
# 5. consider teams missing player data
# - IMT missing ADC and MID, high impact
# - DIG missing ADC medium impact
# - GG missing JG not sure

In [18]:
# Which players playing in Spring 2020 have played
# on a professional team the previous year? Players
# who are new in Spring 2020 will decrease accuracy
# of predictions for their team.

all = []
for team in na:
    played = []
    for player in team:
        if player in new_na['player'].unique():
            played.append([True, player])
        else:
            played.append([False, player])
    all.append(played)
all

[[[True, 'Doublelift'],
  [True, 'Impact'],
  [True, 'Jensen'],
  [True, 'CoreJJ'],
  [True, 'Broxah']],
 [[True, 'Ssumday'],
  [True, 'Meteos'],
  [True, 'Cody Sun'],
  [True, 'Stunt'],
  [True, 'ry0ma']],
 [[True, 'Broken Blade'],
  [True, 'Dardoch'],
  [True, 'Kobbe'],
  [True, 'Bjergsen'],
  [True, 'Biofrost']],
 [[True, 'sOAZ'], [True, 'Xmithie'], [False, 'Altec'], [True, 'Hakuho']],
 [[True, 'V1per'],
  [True, 'Santorin'],
  [True, 'PowerOfEvil'],
  [True, 'WildTurtle'],
  [True, 'IgNar']],
 [[True, 'Hauntzer'],
  [False, 'Closer'],
  [True, 'Goldenglue'],
  [True, 'FBI'],
  [True, 'Keith']],
 [[True, 'Kumo'],
  [True, 'Svenskeren'],
  [True, 'Zeyzal'],
  [True, 'Bang'],
  [True, 'Jiizuke']],
 [[True, 'Ruin'],
  [True, 'Wiggily'],
  [True, 'Crown'],
  [True, 'Stixxay'],
  [True, 'Smoothie']],
 [[True, 'Licorice'],
  [True, 'Blaber'],
  [True, 'Nisqy'],
  [True, 'Zven'],
  [True, 'Vulcan']],
 [[True, 'Huni'],
  [True, 'Grig'],
  [True, 'Froggen'],
  [False, 'Johnsun'],
  [True, 'a

In [19]:
kda_pred = ['TL', 'C9', 'TSM', 'EG', 'FQ', 'IMT', 'CLG', 'GG', '100T', 'DIG']
good_bad = ['TL', 'C9', 'TSM', 'EG', 'CLG', 'IMT', 'DIG', 'FQ', 'GG', '100T']
avg = ['TL', 'C9', 'TSM', 'EG', 'CLG', 'IMT', 'FQ', 'DIG', 'GG', '100T']

In [21]:
# Predictions for Spring 2020 season standings based
# on individual player performance.

wr_pred = []
features1 = good+bad
features2 = ['k', 'd', 'a']
features = features2

for team in na:
    predictions = model.fit(x[features], y).predict(new_na[new_na['player'].isin(team)][features])
    wr = sum(predictions) / len(predictions)
    wr_pred.append([team, wr])
pd.DataFrame(wr_pred).sort_values(by=1, ascending=False)

,0,1
0,"[Doublelift, Impact, Jensen, CoreJJ, Broxah]",0.706093
8,"[Licorice, Blaber, Nisqy, Zven, Vulcan]",0.560538
2,"[Broken Blade, Dardoch, Kobbe, Bjergsen, Biofr...",0.536170
6,"[Kumo, Svenskeren, Zeyzal, Bang, Jiizuke]",0.533708
4,"[V1per, Santorin, PowerOfEvil, WildTurtle, IgNar]",0.520000
3,"[sOAZ, Xmithie, Altec, Hakuho]",0.516667
7,"[Ruin, Wiggily, Crown, Stixxay, Smoothie]",0.482906
5,"[Hauntzer, Closer, Goldenglue, FBI, Keith]",0.456140
1,"[Ssumday, Meteos, Cody Sun, Stunt, ry0ma]",0.445455
9,"[Huni, Grig, Froggen, Johnsun, aphromoo]",0.400000
